In [1]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb

import warnings
warnings.filterwarnings('ignore')

# Task 1 / Preprocessing

In [2]:
train = pd.read_csv('train.csv' , sep = '\t').drop( columns = 'Unnamed: 0')
test = pd.read_csv('test.csv', sep = '\t').drop( columns = 'Unnamed: 0')

print('size :' + str(train.shape))
print('trget % :' + str(train['0'].value_counts(normalize =True)[1]))
train.head()

size :(30500, 346)
trget % :0.17780327868852458


,0,1,2,3,4,5,6,7,8,9,...,336,337,338,339,340,341,342,343,344,345
0,1,1,0,0,0,0,0,0.090909,0,0,...,0.221395,0,1,0,0,0.222222,1,1,1,1
1,1,1,0,0,1,0,0,0.090909,0,0,...,0.241508,0,1,0,0,0.111111,1,1,1,0
2,0,1,0,0,1,0,0,0.090909,0,0,...,0.123067,0,1,0,0,0.444444,1,1,1,1
3,0,1,0,0,1,0,0,0.136364,0,0,...,0.296065,0,0,1,0,0.222222,1,1,1,0
4,0,1,0,0,1,0,0,0.136364,0,0,...,0.178956,0,0,1,0,0.111111,1,1,1,1


In [3]:
for column in train.columns[1:]  :
    delIndex = train.index[ (train[column] < test[column].min()) | (test[column].max() < train[column] )  ]
    train.drop( delIndex , inplace= True )
    
badCol = train.columns[[train[col].unique().size == 1 for col in train.columns ]]   
train.drop(columns = badCol , inplace = True )
test.drop(columns = badCol , inplace = True )
train.reset_index(drop=True , inplace= True)

print('size :' + str(train.shape))
print('trget % :' + str(train['0'].value_counts(normalize =True)[1]))
train.head()

size :(30254, 324)
trget % :0.17805909962319033


,0,1,2,3,4,6,7,8,9,10,...,336,337,338,339,340,341,342,343,344,345
0,1,1,0,0,0,0,0.090909,0,0,1,...,0.221395,0,1,0,0,0.222222,1,1,1,1
1,1,1,0,0,1,0,0.090909,0,0,1,...,0.241508,0,1,0,0,0.111111,1,1,1,0
2,0,1,0,0,1,0,0.090909,0,0,1,...,0.123067,0,1,0,0,0.444444,1,1,1,1
3,0,1,0,0,1,0,0.136364,0,0,1,...,0.296065,0,0,1,0,0.222222,1,1,1,0
4,0,1,0,0,1,0,0.136364,0,0,1,...,0.178956,0,0,1,0,0.111111,1,1,1,1


In [4]:
ppReportTrain = pp.ProfileReport(train)

In [5]:
#ppReportTrain

In [6]:
train.drop(columns = ['9', '153'] , inplace = True )
test.drop(columns = ['9', '153'] , inplace = True )
#corColumns = ppReportTrain.get_rejected_variables()
corColumns = ['117', '119', '122', '128', '146', '148', '153', '158', '165', '179', '180', 
              '206', '323', '324', '325', '326', '327', '333', '334', '335', '341', '9']
corColumns.remove('9')
corColumns.remove('153')

train size : 30254 | test size : 4166

train_size/test_size = 7.262121939510322

train/7 = 4322

4322 +-= 4166

=> 7 split

In [47]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold , cross_val_score , GridSearchCV
X = train.drop(columns = ['0'])
y = train['0'].astype(int)
test_X = test.drop(columns = '0')
skf = StratifiedKFold(n_splits = int(train.shape[0]/test.shape[0] ), shuffle=True, random_state=42)
# пребором (из тех где больше всего уникальных значений)
cat_cols =[ '114', '196', '197', '198', '199',  '201', '202', '203',
           '204', '205', '206', '207', '208', '209', '210','109']

# LightGBM

In [107]:
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
lgb_parameters = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'num_threads': 4,
    "metric": 'auc',
    #'num_leaves': 63,
    #'max_depth':24 ,
    
#regularization
    'colsample_bytree': 0.7,
    'subsample': 0.9,
    'subsample_freq': 1 ,
    'min_data_in_leaf': 60,
    
    'cat_smooth': 10,
    'min_data_per_group': 50
}

lgb_train = lgb.Dataset(X, label=y, free_raw_data=False , categorical_feature=cat_cols)

result = lgb.cv(lgb_parameters, lgb_train, 10000, folds=skf.split(X, y), 
                early_stopping_rounds=100, verbose_eval=1,)

lgb_rounds = pd.Series( result['auc-mean']).idxmax()+1
max(result['auc-mean']) , lgb_rounds

[1]	cv_agg's auc: 0.684205 + 0.0107754
[2]	cv_agg's auc: 0.697242 + 0.00956793
[3]	cv_agg's auc: 0.703371 + 0.00640683
[4]	cv_agg's auc: 0.704805 + 0.00627745
[5]	cv_agg's auc: 0.706185 + 0.00621936
[6]	cv_agg's auc: 0.708371 + 0.00521651
[7]	cv_agg's auc: 0.707998 + 0.00595215
[8]	cv_agg's auc: 0.708338 + 0.0056327
[9]	cv_agg's auc: 0.709917 + 0.00600075
[10]	cv_agg's auc: 0.710252 + 0.00581025
[11]	cv_agg's auc: 0.710834 + 0.00616473
[12]	cv_agg's auc: 0.711281 + 0.00599228
[13]	cv_agg's auc: 0.711869 + 0.00587167
[14]	cv_agg's auc: 0.711879 + 0.00588695
[15]	cv_agg's auc: 0.712153 + 0.00627078
[16]	cv_agg's auc: 0.712648 + 0.0063542
[17]	cv_agg's auc: 0.712978 + 0.00628925
[18]	cv_agg's auc: 0.71313 + 0.00647701
[19]	cv_agg's auc: 0.713251 + 0.00645784
[20]	cv_agg's auc: 0.713429 + 0.00657099
[21]	cv_agg's auc: 0.713817 + 0.00626439
[22]	cv_agg's auc: 0.714076 + 0.00637338
[23]	cv_agg's auc: 0.714067 + 0.00634019
[24]	cv_agg's auc: 0.714054 + 0.00633445
[25]	cv_agg's auc: 0.714052 +

[203]	cv_agg's auc: 0.730369 + 0.00656402
[204]	cv_agg's auc: 0.730413 + 0.0065674
[205]	cv_agg's auc: 0.730508 + 0.00655997
[206]	cv_agg's auc: 0.730658 + 0.00660825
[207]	cv_agg's auc: 0.730723 + 0.00660235
[208]	cv_agg's auc: 0.730838 + 0.00657647
[209]	cv_agg's auc: 0.730906 + 0.00654773
[210]	cv_agg's auc: 0.731016 + 0.00659482
[211]	cv_agg's auc: 0.731128 + 0.0065946
[212]	cv_agg's auc: 0.73123 + 0.00657262
[213]	cv_agg's auc: 0.731327 + 0.00660006
[214]	cv_agg's auc: 0.731429 + 0.00654199
[215]	cv_agg's auc: 0.731547 + 0.0065929
[216]	cv_agg's auc: 0.731559 + 0.00659785
[217]	cv_agg's auc: 0.731703 + 0.00653956
[218]	cv_agg's auc: 0.731861 + 0.00650209
[219]	cv_agg's auc: 0.731917 + 0.00648227
[220]	cv_agg's auc: 0.731985 + 0.00650189
[221]	cv_agg's auc: 0.732102 + 0.00650694
[222]	cv_agg's auc: 0.732181 + 0.00652969
[223]	cv_agg's auc: 0.73223 + 0.00651142
[224]	cv_agg's auc: 0.732329 + 0.00651473
[225]	cv_agg's auc: 0.732479 + 0.00651449
[226]	cv_agg's auc: 0.732591 + 0.006478

[402]	cv_agg's auc: 0.740936 + 0.00701424
[403]	cv_agg's auc: 0.740988 + 0.00699575
[404]	cv_agg's auc: 0.741032 + 0.00699516
[405]	cv_agg's auc: 0.741072 + 0.00699861
[406]	cv_agg's auc: 0.74108 + 0.00699456
[407]	cv_agg's auc: 0.741104 + 0.00700108
[408]	cv_agg's auc: 0.741124 + 0.00702308
[409]	cv_agg's auc: 0.741192 + 0.00700112
[410]	cv_agg's auc: 0.741246 + 0.00701493
[411]	cv_agg's auc: 0.741317 + 0.00698905
[412]	cv_agg's auc: 0.741316 + 0.00698883
[413]	cv_agg's auc: 0.741359 + 0.00697251
[414]	cv_agg's auc: 0.741405 + 0.00700158
[415]	cv_agg's auc: 0.7414 + 0.00699267
[416]	cv_agg's auc: 0.741469 + 0.00699578
[417]	cv_agg's auc: 0.741523 + 0.0069876
[418]	cv_agg's auc: 0.741571 + 0.0069636
[419]	cv_agg's auc: 0.741586 + 0.00695522
[420]	cv_agg's auc: 0.741594 + 0.00697081
[421]	cv_agg's auc: 0.741644 + 0.00696014
[422]	cv_agg's auc: 0.74167 + 0.00696489
[423]	cv_agg's auc: 0.74167 + 0.00696732
[424]	cv_agg's auc: 0.741691 + 0.00696365
[425]	cv_agg's auc: 0.741742 + 0.00699098

[601]	cv_agg's auc: 0.743883 + 0.00724787
[602]	cv_agg's auc: 0.743897 + 0.00722168
[603]	cv_agg's auc: 0.743908 + 0.00725531
[604]	cv_agg's auc: 0.7439 + 0.00725843
[605]	cv_agg's auc: 0.743909 + 0.00724768
[606]	cv_agg's auc: 0.743932 + 0.00725351
[607]	cv_agg's auc: 0.743944 + 0.00722254
[608]	cv_agg's auc: 0.743972 + 0.00723058
[609]	cv_agg's auc: 0.743981 + 0.00719961
[610]	cv_agg's auc: 0.743967 + 0.00721602
[611]	cv_agg's auc: 0.743975 + 0.00720778
[612]	cv_agg's auc: 0.743984 + 0.00725179
[613]	cv_agg's auc: 0.743978 + 0.00723845
[614]	cv_agg's auc: 0.743979 + 0.00723803
[615]	cv_agg's auc: 0.743969 + 0.0072566
[616]	cv_agg's auc: 0.743992 + 0.00728608
[617]	cv_agg's auc: 0.74399 + 0.00725737
[618]	cv_agg's auc: 0.74398 + 0.00723226
[619]	cv_agg's auc: 0.743999 + 0.00723845
[620]	cv_agg's auc: 0.744004 + 0.00726703
[621]	cv_agg's auc: 0.744021 + 0.0072614
[622]	cv_agg's auc: 0.744007 + 0.00726864
[623]	cv_agg's auc: 0.744017 + 0.00727329
[624]	cv_agg's auc: 0.744031 + 0.0072762

[798]	cv_agg's auc: 0.744626 + 0.00732769
[799]	cv_agg's auc: 0.744629 + 0.00734263
[800]	cv_agg's auc: 0.744632 + 0.00735665
[801]	cv_agg's auc: 0.744621 + 0.00736269
[802]	cv_agg's auc: 0.744637 + 0.00737827
[803]	cv_agg's auc: 0.74466 + 0.00736107
[804]	cv_agg's auc: 0.744678 + 0.00736778
[805]	cv_agg's auc: 0.744679 + 0.00736006
[806]	cv_agg's auc: 0.744683 + 0.00733931
[807]	cv_agg's auc: 0.744681 + 0.00736188
[808]	cv_agg's auc: 0.744685 + 0.00735416
[809]	cv_agg's auc: 0.74469 + 0.00733985
[810]	cv_agg's auc: 0.744701 + 0.00734117
[811]	cv_agg's auc: 0.744666 + 0.00733618
[812]	cv_agg's auc: 0.744656 + 0.00732046
[813]	cv_agg's auc: 0.744659 + 0.00731065
[814]	cv_agg's auc: 0.744663 + 0.00731964
[815]	cv_agg's auc: 0.744675 + 0.00732783
[816]	cv_agg's auc: 0.744672 + 0.00733481
[817]	cv_agg's auc: 0.744656 + 0.00733612
[818]	cv_agg's auc: 0.744655 + 0.0073671
[819]	cv_agg's auc: 0.744654 + 0.00735089
[820]	cv_agg's auc: 0.744673 + 0.00734287
[821]	cv_agg's auc: 0.744688 + 0.0073

[999]	cv_agg's auc: 0.745067 + 0.00751987
[1000]	cv_agg's auc: 0.745073 + 0.00752995
[1001]	cv_agg's auc: 0.745056 + 0.00753015
[1002]	cv_agg's auc: 0.745067 + 0.0075156
[1003]	cv_agg's auc: 0.745076 + 0.00752942
[1004]	cv_agg's auc: 0.745073 + 0.00752385
[1005]	cv_agg's auc: 0.745076 + 0.00753149
[1006]	cv_agg's auc: 0.745094 + 0.00752708
[1007]	cv_agg's auc: 0.745094 + 0.00750931
[1008]	cv_agg's auc: 0.745082 + 0.00751934
[1009]	cv_agg's auc: 0.745079 + 0.00751485
[1010]	cv_agg's auc: 0.74511 + 0.0075125
[1011]	cv_agg's auc: 0.745118 + 0.00750143
[1012]	cv_agg's auc: 0.74512 + 0.00749506
[1013]	cv_agg's auc: 0.745104 + 0.00749197
[1014]	cv_agg's auc: 0.745137 + 0.00749134
[1015]	cv_agg's auc: 0.745124 + 0.00748503
[1016]	cv_agg's auc: 0.745125 + 0.00748138
[1017]	cv_agg's auc: 0.745132 + 0.00749694
[1018]	cv_agg's auc: 0.745127 + 0.00751364
[1019]	cv_agg's auc: 0.74513 + 0.00751329
[1020]	cv_agg's auc: 0.745135 + 0.00749986
[1021]	cv_agg's auc: 0.745132 + 0.00748051
[1022]	cv_agg's a

(0.7452086097409014, 1055)

In [33]:
model = lgb.train(lgb_parameters , lgb_train , lgb_rounds, categorical_feature= cat_cols)
pd.Series(model.predict(test.drop(columns = '0'))).to_csv('submission_LGBM.csv',header = True, index_label = True)

lb : 0.76358517

# XgBoost

In [64]:
xgb_parameters = {
    #default
    'objective': 'binary:logistic',
    'eta': 0.01, 
    'silent': 1,
    "nthread": -1,
    "random_seed": 42,
    "eval_metric": 'auc',
    
    # regularization parameters
    #'max_leaves': 63,
    #'max_depth': 22,
    'subsample': 0.8,
    'colsample_bytree': 0.6,
    
    #lightgbm approach
    'tree_method': 'hist',
    'grow_policy': 'lossguide',
    'predictor':'cpu_predictor'
}
xgb_train = xgb.DMatrix(X, y, feature_names=X.columns)

results = xgb.cv(xgb_parameters, xgb_train, 10000, early_stopping_rounds=100,
                 folds=skf, verbose_eval=1 )

xgb_rounds = pd.Series( results['test-auc-mean']).idxmax()+1

max(results['test-auc-mean']) , xgb_rounds

[0]	train-auc:0.694696+0.00203685	test-auc:0.666536+0.00877186
[1]	train-auc:0.722979+0.00324221	test-auc:0.692177+0.00848099
[2]	train-auc:0.733332+0.00275926	test-auc:0.697222+0.00660241
[3]	train-auc:0.737466+0.00101313	test-auc:0.699269+0.00881204
[4]	train-auc:0.740746+0.00327302	test-auc:0.7008+0.00729653
[5]	train-auc:0.742486+0.00308917	test-auc:0.702355+0.00655887
[6]	train-auc:0.744789+0.00459166	test-auc:0.70379+0.006199
[7]	train-auc:0.747655+0.00493014	test-auc:0.705137+0.00588124
[8]	train-auc:0.748346+0.00472878	test-auc:0.705694+0.00615876
[9]	train-auc:0.74964+0.00409774	test-auc:0.70723+0.0065379
[10]	train-auc:0.751127+0.00363002	test-auc:0.708152+0.00597085
[11]	train-auc:0.751651+0.00382328	test-auc:0.708583+0.00555975
[12]	train-auc:0.752053+0.0034196	test-auc:0.708673+0.00560174
[13]	train-auc:0.752469+0.00345531	test-auc:0.7093+0.00534609
[14]	train-auc:0.752897+0.0035031	test-auc:0.709879+0.00527454
[15]	train-auc:0.753913+0.00275027	test-auc:0.710313+0.0053258

[129]	train-auc:0.787112+0.000805181	test-auc:0.722844+0.00559164
[130]	train-auc:0.787353+0.000773303	test-auc:0.723+0.00567645
[131]	train-auc:0.787599+0.000810373	test-auc:0.72303+0.00573614
[132]	train-auc:0.787828+0.000826496	test-auc:0.723133+0.00573501
[133]	train-auc:0.788283+0.000897528	test-auc:0.723248+0.00574323
[134]	train-auc:0.78854+0.000742671	test-auc:0.723354+0.00568722
[135]	train-auc:0.788665+0.000643705	test-auc:0.723498+0.00566684
[136]	train-auc:0.788791+0.00062725	test-auc:0.723551+0.0056298
[137]	train-auc:0.789067+0.000533069	test-auc:0.723621+0.00559138
[138]	train-auc:0.78924+0.000523275	test-auc:0.723669+0.00563007
[139]	train-auc:0.789415+0.000506832	test-auc:0.723688+0.00558015
[140]	train-auc:0.789603+0.000485113	test-auc:0.723754+0.00561862
[141]	train-auc:0.789745+0.000456742	test-auc:0.723863+0.00558677
[142]	train-auc:0.78983+0.000448625	test-auc:0.723874+0.00551656
[143]	train-auc:0.790021+0.000388468	test-auc:0.724057+0.00550015
[144]	train-auc:0.7

[254]	train-auc:0.816597+0.000966092	test-auc:0.730734+0.005921
[255]	train-auc:0.816797+0.00101385	test-auc:0.730791+0.00588912
[256]	train-auc:0.816947+0.00100927	test-auc:0.730841+0.00584671
[257]	train-auc:0.817159+0.000986562	test-auc:0.73087+0.00583102
[258]	train-auc:0.817404+0.000947318	test-auc:0.730923+0.00587187
[259]	train-auc:0.817575+0.00095534	test-auc:0.730976+0.00585165
[260]	train-auc:0.817786+0.000991718	test-auc:0.731004+0.00581763
[261]	train-auc:0.818036+0.000956921	test-auc:0.73107+0.00583235
[262]	train-auc:0.818223+0.000904655	test-auc:0.731076+0.0057994
[263]	train-auc:0.818414+0.000933056	test-auc:0.731182+0.00576609
[264]	train-auc:0.818692+0.000905811	test-auc:0.731239+0.00576904
[265]	train-auc:0.818865+0.00089717	test-auc:0.731272+0.00579367
[266]	train-auc:0.818986+0.000878955	test-auc:0.731317+0.00578067
[267]	train-auc:0.81918+0.000936934	test-auc:0.731359+0.00573874
[268]	train-auc:0.819395+0.000907153	test-auc:0.731416+0.00573967
[269]	train-auc:0.81

[381]	train-auc:0.841249+0.00146833	test-auc:0.735814+0.00549054
[382]	train-auc:0.841391+0.00149292	test-auc:0.735836+0.00542647
[383]	train-auc:0.841514+0.00154332	test-auc:0.735809+0.00543925
[384]	train-auc:0.841645+0.00148841	test-auc:0.735804+0.00544203
[385]	train-auc:0.841818+0.0015602	test-auc:0.735823+0.00541988
[386]	train-auc:0.84196+0.00157614	test-auc:0.73584+0.00537184
[387]	train-auc:0.842146+0.00153566	test-auc:0.735877+0.00541196
[388]	train-auc:0.842337+0.00160177	test-auc:0.735916+0.00541375
[389]	train-auc:0.84245+0.00166006	test-auc:0.735953+0.00542395
[390]	train-auc:0.842624+0.0016884	test-auc:0.735935+0.00539143
[391]	train-auc:0.842808+0.00177759	test-auc:0.735949+0.00534547
[392]	train-auc:0.843008+0.00182584	test-auc:0.735999+0.00536228
[393]	train-auc:0.843132+0.00184804	test-auc:0.736013+0.00533741
[394]	train-auc:0.843312+0.00182953	test-auc:0.736047+0.00532358
[395]	train-auc:0.843455+0.00178225	test-auc:0.736073+0.00529353
[396]	train-auc:0.843605+0.001

[508]	train-auc:0.861058+0.00183686	test-auc:0.737838+0.00501041
[509]	train-auc:0.86113+0.00184697	test-auc:0.737853+0.0050005
[510]	train-auc:0.861272+0.00188735	test-auc:0.737895+0.00498629
[511]	train-auc:0.861347+0.00190537	test-auc:0.737907+0.00498449
[512]	train-auc:0.861526+0.00195867	test-auc:0.73792+0.00497032
[513]	train-auc:0.861652+0.00196837	test-auc:0.737922+0.00494744
[514]	train-auc:0.861799+0.001937	test-auc:0.737945+0.00492892
[515]	train-auc:0.861908+0.0019523	test-auc:0.737977+0.00493311
[516]	train-auc:0.862111+0.00202213	test-auc:0.738041+0.00495409
[517]	train-auc:0.862299+0.00207737	test-auc:0.738075+0.00495273
[518]	train-auc:0.862454+0.00199276	test-auc:0.738119+0.00492858
[519]	train-auc:0.862554+0.00200193	test-auc:0.738121+0.00491053
[520]	train-auc:0.862714+0.00198687	test-auc:0.73814+0.00488436
[521]	train-auc:0.862843+0.00198204	test-auc:0.738155+0.00486081
[522]	train-auc:0.862995+0.00194638	test-auc:0.738159+0.00485022
[523]	train-auc:0.863153+0.00190

[635]	train-auc:0.877253+0.00177809	test-auc:0.739258+0.00457925
[636]	train-auc:0.87734+0.00179895	test-auc:0.739265+0.00455813
[637]	train-auc:0.87748+0.00177881	test-auc:0.739257+0.00453988
[638]	train-auc:0.8776+0.00176576	test-auc:0.739284+0.00451922
[639]	train-auc:0.877716+0.00176419	test-auc:0.739305+0.00450928
[640]	train-auc:0.877803+0.00175256	test-auc:0.739321+0.00452694
[641]	train-auc:0.877918+0.00175073	test-auc:0.739327+0.00451584
[642]	train-auc:0.877982+0.00173627	test-auc:0.739343+0.00452881
[643]	train-auc:0.878067+0.00178226	test-auc:0.739357+0.0045229
[644]	train-auc:0.878176+0.00177721	test-auc:0.73935+0.00453148
[645]	train-auc:0.878316+0.00175692	test-auc:0.739351+0.00454288
[646]	train-auc:0.87843+0.00172198	test-auc:0.739392+0.00453097
[647]	train-auc:0.878633+0.00169944	test-auc:0.739406+0.00455728
[648]	train-auc:0.878796+0.00171998	test-auc:0.739421+0.00456814
[649]	train-auc:0.878903+0.00176411	test-auc:0.739412+0.00454651
[650]	train-auc:0.879006+0.00175

[762]	train-auc:0.892107+0.00130409	test-auc:0.740224+0.00420132
[763]	train-auc:0.892218+0.00133222	test-auc:0.740226+0.00420419
[764]	train-auc:0.892312+0.00133797	test-auc:0.740244+0.00418128
[765]	train-auc:0.89244+0.00133798	test-auc:0.740255+0.00418266
[766]	train-auc:0.892551+0.00135008	test-auc:0.740235+0.00421427
[767]	train-auc:0.892645+0.00133889	test-auc:0.740223+0.00418994
[768]	train-auc:0.892751+0.00134941	test-auc:0.74021+0.00420593
[769]	train-auc:0.892839+0.00136391	test-auc:0.740201+0.00422159
[770]	train-auc:0.892888+0.00136229	test-auc:0.740206+0.00421112
[771]	train-auc:0.89294+0.00135072	test-auc:0.740212+0.00419773
[772]	train-auc:0.893021+0.00135127	test-auc:0.740199+0.0042054
[773]	train-auc:0.893107+0.00131252	test-auc:0.740223+0.00421842
[774]	train-auc:0.89321+0.00134924	test-auc:0.740242+0.00422865
[775]	train-auc:0.893318+0.00130984	test-auc:0.740244+0.00423639
[776]	train-auc:0.893416+0.00130393	test-auc:0.740238+0.00421058
[777]	train-auc:0.89349+0.0012

[889]	train-auc:0.904774+0.00129613	test-auc:0.740551+0.00392933
[890]	train-auc:0.904853+0.00131101	test-auc:0.740567+0.00392067
[891]	train-auc:0.904972+0.00133045	test-auc:0.74056+0.00390627
[892]	train-auc:0.905063+0.00139213	test-auc:0.740558+0.00390118
[893]	train-auc:0.905129+0.00139931	test-auc:0.74055+0.0038873
[894]	train-auc:0.905217+0.00136124	test-auc:0.740544+0.00388125
[895]	train-auc:0.905307+0.0013362	test-auc:0.740525+0.00388042
[896]	train-auc:0.905431+0.00135931	test-auc:0.740544+0.00387059
[897]	train-auc:0.905497+0.00134461	test-auc:0.740513+0.00383987
[898]	train-auc:0.905563+0.00131259	test-auc:0.740521+0.00384352
[899]	train-auc:0.905672+0.00130782	test-auc:0.740523+0.00383715
[900]	train-auc:0.905768+0.00135876	test-auc:0.740502+0.00386476
[901]	train-auc:0.905847+0.00133874	test-auc:0.740509+0.00386155
[902]	train-auc:0.905904+0.00135444	test-auc:0.740507+0.00386812
[903]	train-auc:0.905988+0.0013622	test-auc:0.740499+0.00387485
[904]	train-auc:0.906111+0.001

(0.7406326666666666, 880)

In [66]:
xgb_test = xgb.DMatrix(X, feature_names=X.loc[val].columns)
model = xgb.train(xgb_parameters, xgb_train, xgb_rounds,verbose_eval=1 )
pd.Series(model.predict(xgb_test)).to_csv('submission_XGB.csv',header = True, index_label = True)

lb : 0.762228

# CatBoost

In [8]:
ctb_parameters = {
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 10000,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': False,
    'od_wait': 30,
    'od_type': 'Iter' ,
        #regularization
    'rsm': 0.9,
    'l2_leaf_reg': 2,
    #'subsample' : 0.8
    #'bagging_temperature': 0.8
}
ctb_data = ctb.Pool(X,y  )
result = ctb.cv(ctb_data, ctb_parameters, stratified=True, seed=42, logging_level=None)

0:	learn: 0.6214568	test: 0.6231337	best: 0.6231337 (0)	total: 179ms	remaining: 29m 51s
1:	learn: 0.6399403	test: 0.6393981	best: 0.6393981 (1)	total: 305ms	remaining: 25m 25s
2:	learn: 0.6538612	test: 0.6540883	best: 0.6540883 (2)	total: 486ms	remaining: 27m
3:	learn: 0.6591375	test: 0.6572077	best: 0.6572077 (3)	total: 665ms	remaining: 27m 41s
4:	learn: 0.6672722	test: 0.6655765	best: 0.6655765 (4)	total: 869ms	remaining: 28m 57s
5:	learn: 0.6689841	test: 0.6677982	best: 0.6677982 (5)	total: 1.04s	remaining: 28m 56s
6:	learn: 0.6717734	test: 0.6694431	best: 0.6694431 (6)	total: 1.24s	remaining: 29m 24s
7:	learn: 0.6705626	test: 0.6681210	best: 0.6694431 (6)	total: 1.4s	remaining: 29m 10s
8:	learn: 0.6741889	test: 0.6707161	best: 0.6707161 (8)	total: 1.57s	remaining: 28m 59s
9:	learn: 0.6762030	test: 0.6722283	best: 0.6722283 (9)	total: 1.74s	remaining: 28m 57s
10:	learn: 0.6790376	test: 0.6736374	best: 0.6736374 (10)	total: 1.96s	remaining: 29m 43s
11:	learn: 0.6794063	test: 0.672861

92:	learn: 0.7325033	test: 0.7148553	best: 0.7148553 (92)	total: 15.6s	remaining: 27m 42s
93:	learn: 0.7328328	test: 0.7149785	best: 0.7149785 (93)	total: 15.8s	remaining: 27m 44s
94:	learn: 0.7330986	test: 0.7151204	best: 0.7151204 (94)	total: 16s	remaining: 27m 44s
95:	learn: 0.7334514	test: 0.7154481	best: 0.7154481 (95)	total: 16.1s	remaining: 27m 42s
96:	learn: 0.7338479	test: 0.7157154	best: 0.7157154 (96)	total: 16.3s	remaining: 27m 45s
97:	learn: 0.7341906	test: 0.7159242	best: 0.7159242 (97)	total: 16.5s	remaining: 27m 44s
98:	learn: 0.7344317	test: 0.7159902	best: 0.7159902 (98)	total: 16.6s	remaining: 27m 42s
99:	learn: 0.7347225	test: 0.7162137	best: 0.7162137 (99)	total: 16.8s	remaining: 27m 40s
100:	learn: 0.7350357	test: 0.7163189	best: 0.7163189 (100)	total: 16.9s	remaining: 27m 40s
101:	learn: 0.7352361	test: 0.7165172	best: 0.7165172 (101)	total: 17.1s	remaining: 27m 38s
102:	learn: 0.7356521	test: 0.7167194	best: 0.7167194 (102)	total: 17.3s	remaining: 27m 38s
103:	l

183:	learn: 0.7558570	test: 0.7266359	best: 0.7266359 (183)	total: 30.6s	remaining: 27m 13s
184:	learn: 0.7560669	test: 0.7267412	best: 0.7267412 (184)	total: 30.8s	remaining: 27m 13s
185:	learn: 0.7563395	test: 0.7268431	best: 0.7268431 (185)	total: 30.9s	remaining: 27m 12s
186:	learn: 0.7564926	test: 0.7269214	best: 0.7269214 (186)	total: 31.1s	remaining: 27m 11s
187:	learn: 0.7568186	test: 0.7270345	best: 0.7270345 (187)	total: 31.3s	remaining: 27m 11s
188:	learn: 0.7570695	test: 0.7271618	best: 0.7271618 (188)	total: 31.4s	remaining: 27m 10s
189:	learn: 0.7572676	test: 0.7272426	best: 0.7272426 (189)	total: 31.6s	remaining: 27m 9s
190:	learn: 0.7574746	test: 0.7273643	best: 0.7273643 (190)	total: 31.7s	remaining: 27m 9s
191:	learn: 0.7576121	test: 0.7274200	best: 0.7274200 (191)	total: 31.9s	remaining: 27m 8s
192:	learn: 0.7577445	test: 0.7274876	best: 0.7274876 (192)	total: 32s	remaining: 27m 8s
193:	learn: 0.7579039	test: 0.7275235	best: 0.7275235 (193)	total: 32.2s	remaining: 27

273:	learn: 0.7725219	test: 0.7319219	best: 0.7319392 (272)	total: 45s	remaining: 26m 37s
274:	learn: 0.7726645	test: 0.7319882	best: 0.7319882 (274)	total: 45.2s	remaining: 26m 37s
275:	learn: 0.7727813	test: 0.7320619	best: 0.7320619 (275)	total: 45.3s	remaining: 26m 36s
276:	learn: 0.7729199	test: 0.7320889	best: 0.7320889 (276)	total: 45.5s	remaining: 26m 36s
277:	learn: 0.7731720	test: 0.7322134	best: 0.7322134 (277)	total: 45.6s	remaining: 26m 36s
278:	learn: 0.7733525	test: 0.7322480	best: 0.7322480 (278)	total: 45.8s	remaining: 26m 35s
279:	learn: 0.7735030	test: 0.7322883	best: 0.7322883 (279)	total: 46s	remaining: 26m 35s
280:	learn: 0.7736146	test: 0.7322954	best: 0.7322954 (280)	total: 46.1s	remaining: 26m 34s
281:	learn: 0.7738301	test: 0.7322617	best: 0.7322954 (280)	total: 46.3s	remaining: 26m 34s
282:	learn: 0.7739866	test: 0.7323551	best: 0.7323551 (282)	total: 46.4s	remaining: 26m 33s
283:	learn: 0.7741969	test: 0.7323703	best: 0.7323703 (283)	total: 46.6s	remaining: 

363:	learn: 0.7890258	test: 0.7354236	best: 0.7354236 (363)	total: 59.5s	remaining: 26m 14s
364:	learn: 0.7892780	test: 0.7354260	best: 0.7354260 (364)	total: 59.6s	remaining: 26m 13s
365:	learn: 0.7895166	test: 0.7354549	best: 0.7354549 (365)	total: 59.8s	remaining: 26m 13s
366:	learn: 0.7896882	test: 0.7354080	best: 0.7354549 (365)	total: 59.9s	remaining: 26m 13s
367:	learn: 0.7898372	test: 0.7354272	best: 0.7354549 (365)	total: 1m	remaining: 26m 12s
368:	learn: 0.7900151	test: 0.7354931	best: 0.7354931 (368)	total: 1m	remaining: 26m 12s
369:	learn: 0.7901795	test: 0.7355316	best: 0.7355316 (369)	total: 1m	remaining: 26m 12s
370:	learn: 0.7903627	test: 0.7355325	best: 0.7355325 (370)	total: 1m	remaining: 26m 11s
371:	learn: 0.7905464	test: 0.7355576	best: 0.7355576 (371)	total: 1m	remaining: 26m 11s
372:	learn: 0.7907028	test: 0.7355447	best: 0.7355576 (371)	total: 1m	remaining: 26m 11s
373:	learn: 0.7909535	test: 0.7355622	best: 0.7355622 (373)	total: 1m 1s	remaining: 26m 10s
374:	l

453:	learn: 0.8053708	test: 0.7369773	best: 0.7369773 (453)	total: 1m 14s	remaining: 25m 59s
454:	learn: 0.8055585	test: 0.7370090	best: 0.7370090 (454)	total: 1m 14s	remaining: 25m 59s
455:	learn: 0.8056894	test: 0.7370494	best: 0.7370494 (455)	total: 1m 14s	remaining: 25m 59s
456:	learn: 0.8058155	test: 0.7370134	best: 0.7370494 (455)	total: 1m 14s	remaining: 25m 58s
457:	learn: 0.8060470	test: 0.7370228	best: 0.7370494 (455)	total: 1m 14s	remaining: 25m 58s
458:	learn: 0.8062329	test: 0.7370719	best: 0.7370719 (458)	total: 1m 14s	remaining: 25m 58s
459:	learn: 0.8063558	test: 0.7371324	best: 0.7371324 (459)	total: 1m 15s	remaining: 25m 58s
460:	learn: 0.8064458	test: 0.7371131	best: 0.7371324 (459)	total: 1m 15s	remaining: 25m 58s
461:	learn: 0.8066521	test: 0.7371041	best: 0.7371324 (459)	total: 1m 15s	remaining: 25m 57s
462:	learn: 0.8068122	test: 0.7371354	best: 0.7371354 (462)	total: 1m 15s	remaining: 25m 58s
463:	learn: 0.8069555	test: 0.7371013	best: 0.7371354 (462)	total: 1m 

543:	learn: 0.8189715	test: 0.7382285	best: 0.7382285 (543)	total: 1m 28s	remaining: 25m 40s
544:	learn: 0.8191667	test: 0.7382519	best: 0.7382519 (544)	total: 1m 28s	remaining: 25m 40s
545:	learn: 0.8193524	test: 0.7382872	best: 0.7382872 (545)	total: 1m 29s	remaining: 25m 41s
546:	learn: 0.8194723	test: 0.7383110	best: 0.7383110 (546)	total: 1m 29s	remaining: 25m 41s
547:	learn: 0.8196138	test: 0.7383204	best: 0.7383204 (547)	total: 1m 29s	remaining: 25m 40s
548:	learn: 0.8197843	test: 0.7383173	best: 0.7383204 (547)	total: 1m 29s	remaining: 25m 40s
549:	learn: 0.8199269	test: 0.7383804	best: 0.7383804 (549)	total: 1m 29s	remaining: 25m 40s
550:	learn: 0.8200926	test: 0.7384544	best: 0.7384544 (550)	total: 1m 29s	remaining: 25m 40s
551:	learn: 0.8202231	test: 0.7385002	best: 0.7385002 (551)	total: 1m 29s	remaining: 25m 39s
552:	learn: 0.8204283	test: 0.7385244	best: 0.7385244 (552)	total: 1m 30s	remaining: 25m 39s
553:	learn: 0.8205635	test: 0.7385509	best: 0.7385509 (553)	total: 1m 

633:	learn: 0.8313210	test: 0.7391934	best: 0.7392019 (632)	total: 1m 43s	remaining: 25m 33s
634:	learn: 0.8314432	test: 0.7391804	best: 0.7392019 (632)	total: 1m 43s	remaining: 25m 33s
635:	learn: 0.8316331	test: 0.7392055	best: 0.7392055 (635)	total: 1m 44s	remaining: 25m 32s
636:	learn: 0.8317782	test: 0.7392269	best: 0.7392269 (636)	total: 1m 44s	remaining: 25m 32s
637:	learn: 0.8319005	test: 0.7392255	best: 0.7392269 (636)	total: 1m 44s	remaining: 25m 32s
638:	learn: 0.8320520	test: 0.7392417	best: 0.7392417 (638)	total: 1m 44s	remaining: 25m 32s
639:	learn: 0.8321517	test: 0.7392353	best: 0.7392417 (638)	total: 1m 44s	remaining: 25m 32s
640:	learn: 0.8322891	test: 0.7392615	best: 0.7392615 (640)	total: 1m 44s	remaining: 25m 32s
641:	learn: 0.8324671	test: 0.7392789	best: 0.7392789 (641)	total: 1m 45s	remaining: 25m 31s
642:	learn: 0.8326357	test: 0.7392739	best: 0.7392789 (641)	total: 1m 45s	remaining: 25m 31s
643:	learn: 0.8327462	test: 0.7392910	best: 0.7392910 (643)	total: 1m 

722:	learn: 0.8426990	test: 0.7399283	best: 0.7399490 (716)	total: 1m 58s	remaining: 25m 18s
723:	learn: 0.8428266	test: 0.7399344	best: 0.7399490 (716)	total: 1m 58s	remaining: 25m 18s
724:	learn: 0.8429047	test: 0.7399453	best: 0.7399490 (716)	total: 1m 58s	remaining: 25m 18s
725:	learn: 0.8430486	test: 0.7399678	best: 0.7399678 (725)	total: 1m 58s	remaining: 25m 17s
726:	learn: 0.8431562	test: 0.7399629	best: 0.7399678 (725)	total: 1m 58s	remaining: 25m 17s
727:	learn: 0.8433473	test: 0.7399385	best: 0.7399678 (725)	total: 1m 59s	remaining: 25m 17s
728:	learn: 0.8434602	test: 0.7399587	best: 0.7399678 (725)	total: 1m 59s	remaining: 25m 17s
729:	learn: 0.8435788	test: 0.7399829	best: 0.7399829 (729)	total: 1m 59s	remaining: 25m 16s
730:	learn: 0.8436688	test: 0.7399943	best: 0.7399943 (730)	total: 1m 59s	remaining: 25m 16s
731:	learn: 0.8437828	test: 0.7399866	best: 0.7399943 (730)	total: 1m 59s	remaining: 25m 16s
732:	learn: 0.8439073	test: 0.7400050	best: 0.7400050 (732)	total: 1m 

In [49]:
ctb_model = ctb.train(ctb_data, parameters,iterations=765,verbose=False)
pred = ctb_model.predict(test_X , prediction_type= 'Probability')[:,1]
pred.to_csv('submission_CTB',header = True, index_label = True)

0:	learn: 0.6403475	total: 84.7ms	remaining: 59.2s
1:	learn: 0.6594136	total: 157ms	remaining: 54.8s
2:	learn: 0.6589874	total: 227ms	remaining: 52.8s
3:	learn: 0.6639446	total: 308ms	remaining: 53.5s
4:	learn: 0.6738206	total: 385ms	remaining: 53.5s
5:	learn: 0.6729976	total: 455ms	remaining: 52.6s
6:	learn: 0.6756286	total: 534ms	remaining: 52.8s
7:	learn: 0.6759655	total: 617ms	remaining: 53.4s
8:	learn: 0.6779468	total: 693ms	remaining: 53.2s
9:	learn: 0.6778088	total: 775ms	remaining: 53.5s
10:	learn: 0.6777956	total: 854ms	remaining: 53.5s
11:	learn: 0.6817536	total: 936ms	remaining: 53.7s
12:	learn: 0.6808259	total: 1.01s	remaining: 53.6s
13:	learn: 0.6818024	total: 1.1s	remaining: 53.8s
14:	learn: 0.6811824	total: 1.17s	remaining: 53.6s
15:	learn: 0.6814426	total: 1.26s	remaining: 53.8s
16:	learn: 0.6832883	total: 1.34s	remaining: 53.9s
17:	learn: 0.6843248	total: 1.41s	remaining: 53.5s
18:	learn: 0.6862943	total: 1.49s	remaining: 53.6s
19:	learn: 0.6875832	total: 1.57s	remaini

163:	learn: 0.7468571	total: 13.2s	remaining: 43.1s
164:	learn: 0.7470414	total: 13.3s	remaining: 43.1s
165:	learn: 0.7471565	total: 13.4s	remaining: 43s
166:	learn: 0.7474127	total: 13.4s	remaining: 42.9s
167:	learn: 0.7476598	total: 13.5s	remaining: 42.8s
168:	learn: 0.7477665	total: 13.6s	remaining: 42.7s
169:	learn: 0.7479021	total: 13.7s	remaining: 42.6s
170:	learn: 0.7480476	total: 13.7s	remaining: 42.5s
171:	learn: 0.7482231	total: 13.8s	remaining: 42.4s
172:	learn: 0.7484220	total: 13.9s	remaining: 42.3s
173:	learn: 0.7485373	total: 14s	remaining: 42.3s
174:	learn: 0.7487297	total: 14.1s	remaining: 42.2s
175:	learn: 0.7489290	total: 14.1s	remaining: 42.1s
176:	learn: 0.7490970	total: 14.2s	remaining: 42s
177:	learn: 0.7492815	total: 14.3s	remaining: 41.9s
178:	learn: 0.7495301	total: 14.4s	remaining: 41.8s
179:	learn: 0.7499372	total: 14.5s	remaining: 41.8s
180:	learn: 0.7502658	total: 14.5s	remaining: 41.7s
181:	learn: 0.7503842	total: 14.6s	remaining: 41.6s
182:	learn: 0.7506

322:	learn: 0.7701496	total: 26.2s	remaining: 30.5s
323:	learn: 0.7702504	total: 26.2s	remaining: 30.4s
324:	learn: 0.7703676	total: 26.3s	remaining: 30.4s
325:	learn: 0.7705890	total: 26.4s	remaining: 30.3s
326:	learn: 0.7706909	total: 26.5s	remaining: 30.2s
327:	learn: 0.7707879	total: 26.6s	remaining: 30.1s
328:	learn: 0.7708964	total: 26.6s	remaining: 30s
329:	learn: 0.7710613	total: 26.7s	remaining: 30s
330:	learn: 0.7711551	total: 26.8s	remaining: 29.9s
331:	learn: 0.7712243	total: 26.9s	remaining: 29.8s
332:	learn: 0.7713852	total: 27s	remaining: 29.7s
333:	learn: 0.7715921	total: 27s	remaining: 29.6s
334:	learn: 0.7716684	total: 27.1s	remaining: 29.5s
335:	learn: 0.7717855	total: 27.2s	remaining: 29.4s
336:	learn: 0.7719529	total: 27.3s	remaining: 29.4s
337:	learn: 0.7721005	total: 27.3s	remaining: 29.3s
338:	learn: 0.7723845	total: 27.4s	remaining: 29.2s
339:	learn: 0.7725169	total: 27.5s	remaining: 29.1s
340:	learn: 0.7726701	total: 27.6s	remaining: 29s
341:	learn: 0.7728167	

481:	learn: 0.7928331	total: 38.9s	remaining: 17.6s
482:	learn: 0.7929251	total: 39s	remaining: 17.5s
483:	learn: 0.7929966	total: 39.1s	remaining: 17.5s
484:	learn: 0.7930947	total: 39.2s	remaining: 17.4s
485:	learn: 0.7931933	total: 39.3s	remaining: 17.3s
486:	learn: 0.7932752	total: 39.5s	remaining: 17.3s
487:	learn: 0.7933807	total: 39.5s	remaining: 17.2s
488:	learn: 0.7935067	total: 39.6s	remaining: 17.1s
489:	learn: 0.7935259	total: 39.7s	remaining: 17s
490:	learn: 0.7936652	total: 39.8s	remaining: 16.9s
491:	learn: 0.7938211	total: 39.9s	remaining: 16.9s
492:	learn: 0.7939763	total: 39.9s	remaining: 16.8s
493:	learn: 0.7940254	total: 40s	remaining: 16.7s
494:	learn: 0.7941264	total: 40.1s	remaining: 16.6s
495:	learn: 0.7942924	total: 40.2s	remaining: 16.5s
496:	learn: 0.7944654	total: 40.3s	remaining: 16.4s
497:	learn: 0.7946441	total: 40.3s	remaining: 16.4s
498:	learn: 0.7947387	total: 40.4s	remaining: 16.3s
499:	learn: 0.7948591	total: 40.5s	remaining: 16.2s
500:	learn: 0.7950

642:	learn: 0.8115824	total: 52.1s	remaining: 4.62s
643:	learn: 0.8116941	total: 52.2s	remaining: 4.54s
644:	learn: 0.8117815	total: 52.3s	remaining: 4.46s
645:	learn: 0.8118725	total: 52.4s	remaining: 4.38s
646:	learn: 0.8119758	total: 52.5s	remaining: 4.3s
647:	learn: 0.8121008	total: 52.6s	remaining: 4.22s
648:	learn: 0.8122785	total: 52.6s	remaining: 4.14s
649:	learn: 0.8123816	total: 52.7s	remaining: 4.05s
650:	learn: 0.8124950	total: 52.8s	remaining: 3.97s
651:	learn: 0.8126529	total: 52.9s	remaining: 3.89s
652:	learn: 0.8127314	total: 53s	remaining: 3.81s
653:	learn: 0.8128330	total: 53s	remaining: 3.73s
654:	learn: 0.8130069	total: 53.1s	remaining: 3.65s
655:	learn: 0.8131143	total: 53.2s	remaining: 3.57s
656:	learn: 0.8132386	total: 53.3s	remaining: 3.49s
657:	learn: 0.8133673	total: 53.4s	remaining: 3.4s
658:	learn: 0.8134684	total: 53.4s	remaining: 3.32s
659:	learn: 0.8135816	total: 53.5s	remaining: 3.24s
660:	learn: 0.8136653	total: 53.6s	remaining: 3.16s
661:	learn: 0.8137

lb : 0.761139

# Ensemble

In [48]:
train_kfpred = pd.DataFrame(index = train.index)
test_pred = pd.DataFrame(index = test.index)

###### lgb

In [54]:
lgb_parameters = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'num_threads': 4,
    "metric": 'auc',
    'num_leaves': 63,
    'max_depth':25 ,
    
#regularization
    'colsample_bytree': 0.6,
    'subsample': 0.8,
    'subsample_freq': 1 ,
    'min_data_in_leaf': 30,
    
    'cat_smooth': 10,
    'min_data_per_group': 50
}
lgb_data = lgb.Dataset(X, label=y , free_raw_data=False ,categorical_feature =  cat_cols)
lgb_rounds = 787

In [103]:
train_kfpred['lgb'] = pd.Series(index = train.index)
for train_val, val in skf.split(X,y): 
    temp_lgb_train = lgb.Dataset(X.loc[train_val], label=y.loc[train_val] , free_raw_data=False ,categorical_feature =  cat_cols)
    temp_model = lgb.train(lgb_parameters , temp_lgb_train , lgb_rounds,# verbose_eval=lgb_rounds , 
                           categorical_feature= cat_cols)
    train_kfpred['lgb'].loc[val]  = temp_model.predict(X.loc[val]  )
roc_auc_score(y , train_kfpred['lgb'])

0.745526863765427

In [49]:
model = lgb.train(lgb_parameters , lgb_data , lgb_rounds, categorical_feature= cat_cols)
test_pred['lgb'] = model.predict(test_X)

##### xgb

In [56]:
train_kfpred['xgb'] = pd.Series(index = train.index)
xgb_train = xgb.DMatrix(X, y, feature_names=X.columns)
xgb_rounds = 900
xgb_parameters = {
    #default
    'objective': 'binary:logistic',
    'eta': 0.01, 
    'silent': 1,
    "nthread": -1,
    "random_seed": 42,
    "eval_metric": 'auc',
    
    # regularization parameters
    #'max_leaves': 20,
    #'max_depth': 25,
    'subsample': 0.75,
    'colsample_bytree': 0.6,
    
    #lightgbm approach
    'tree_method': 'hist',
    'grow_policy': 'lossguide',
    'predictor':'cpu_predictor'
}

In [57]:
for train_val, val in skf.split(X,y):  
    temp_xgb_train = xgb.DMatrix(X.loc[train_val], y.loc[train_val], feature_names=X.loc[train_val].columns)
    temp_xgb_val = xgb.DMatrix(X.loc[val], y.loc[val], feature_names=X.loc[val].columns)
    temp_model = xgb.train( xgb_parameters,temp_xgb_train, xgb_rounds )
    train_kfpred['xgb'].iloc[val]  = temp_model.predict(temp_xgb_val)
roc_auc_score(y , train_kfpred['xgb'])

0.7442440861678915

In [50]:
xgb_test = xgb.DMatrix(test_X)
temp_model = xgb.train( xgb_parameters, xgb_train, xgb_rounds)
test_pred['xgb']  = temp_model.predict(xgb_test)

##### ctb

In [58]:
train_kfpred['ctb'] = pd.Series(index = train.index)
ctb_train = ctb.Pool(X, y )
ctb_rounds = 765
ctb_parameters = {
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': ctb_rounds,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': False,
 
        #regularization
    'rsm': 0.9,
    'l2_leaf_reg': 2,
    #'subsample' : 0.8
   #'bagging_temperature': 0.8
}

In [59]:
for train_val, val in skf.split(X,y):    
    temp_ctb_train = ctb.Pool(X.loc[train_val], y.loc[train_val])
    temp_ctb_val = ctb.Pool(X.loc[val], y.loc[val] )
    temp_model = ctb.train(temp_ctb_train, ctb_parameters  , verbose=False )
    train_kfpred['ctb'].iloc[val]  = temp_model.predict(temp_ctb_val , prediction_type = 'Probability')[:,1]

roc_auc_score(y , train_kfpred['ctb'])

0.7424837279304553

In [51]:
ctb_model = ctb.train(ctb_train, ctb_parameters, iterations=ctb_rounds ,verbose=False)
test_pred['ctb'] = ctb_model.predict(test_X , prediction_type= 'Probability')[:,1]

---

In [60]:
train_kfpred.head()

,lgb,xgb,ctb
0,0.548268,0.510532,0.527391
1,0.175968,0.214455,0.205134
2,0.231790,0.204278,0.191363
3,0.223408,0.227157,0.261869
4,0.661263,0.596296,0.558705


In [61]:
test_pred.head()

,lgb,xgb,ctb
0,0.155406,0.131656,0.126035
1,0.526920,0.489307,0.555711
2,0.305057,0.279704,0.196673
3,0.325126,0.317252,0.325163
4,0.714722,0.695872,0.528364


###### model single score

In [63]:
for model in train_kfpred.columns:
    print(model + ' :' + str(roc_auc_score(y , train_kfpred[model] )))

lgb :0.7464129364991757
xgb :0.7442440861678915
ctb :0.7424837279304553


###### Simple averaging


In [64]:
from scipy.stats import rankdata
roc_auc_score(y, sum(train_kfpred[model] for model in train_kfpred.columns )/train_kfpred.shape[0] ) 

0.7467841185386562

###### Geometric mean

In [65]:
from functools import reduce
roc_auc_score(y, np.sqrt(train_kfpred['lgb']*train_kfpred['xgb']*train_kfpred['ctb'] ) ) 

0.7468218988878268

###### Rank average

In [66]:
from scipy.stats import rankdata
roc_auc_score(y, sum(rankdata(train_kfpred[el]) for el in train_kfpred) ) 

0.746739530112338

###### Weighted average


In [67]:
roc_auc_score(y, train_kfpred['lgb']*0.65 + train_kfpred['xgb']*0.15 + train_kfpred['ctb']*0.2  )

0.7472939852900295

In [72]:
pred = (test_pred['lgb']*0.65 + test_pred['xgb']*0.15 + test_pred['ctb']*0.2 )
pred.to_csv('submission_Ensemble.csv',header = True, index_label = True)

lb:0.76517585

# Staking

In [81]:
X_st = pd.concat([X , train_kfpred] , axis= 1 )
test_X_st = pd.concat([test_X , test_pred] , axis = 1)

In [102]:
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
lgb_parameters_st = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'num_threads': -1,
    "metric": 'auc',
    #'num_leaves': 63,
    #'max_depth':24 ,
    
#regularization
    'colsample_bytree': 0.8,
    'subsample': 0.6,
    'subsample_freq': 1 ,
    #'min_data_in_leaf': 30,
    
    'cat_smooth': 10,
    'min_data_per_group': 50
}

lgb_train_st = lgb.Dataset(X_st, label=y, free_raw_data=False , categorical_feature=cat_cols)

result = lgb.cv(lgb_parameters_st, lgb_train_st, 10000, folds=skf.split(X_st, y), 
                early_stopping_rounds=100, verbose_eval=1,)

lgb_st_rounds = pd.Series( result['auc-mean']).idxmax()+1
max(result['auc-mean']) , lgb_rounds

[1]	cv_agg's auc: 0.730093 + 0.00665296
[2]	cv_agg's auc: 0.738156 + 0.00742796
[3]	cv_agg's auc: 0.740632 + 0.00706246
[4]	cv_agg's auc: 0.742237 + 0.00698753
[5]	cv_agg's auc: 0.742522 + 0.007232
[6]	cv_agg's auc: 0.742835 + 0.00727675
[7]	cv_agg's auc: 0.74336 + 0.00728589
[8]	cv_agg's auc: 0.743381 + 0.00749591
[9]	cv_agg's auc: 0.743389 + 0.00745176
[10]	cv_agg's auc: 0.743483 + 0.00742956
[11]	cv_agg's auc: 0.74382 + 0.00741822
[12]	cv_agg's auc: 0.743738 + 0.00739557
[13]	cv_agg's auc: 0.743959 + 0.00722748
[14]	cv_agg's auc: 0.743947 + 0.00731708
[15]	cv_agg's auc: 0.743892 + 0.00729192
[16]	cv_agg's auc: 0.744123 + 0.00724097
[17]	cv_agg's auc: 0.744103 + 0.00710807
[18]	cv_agg's auc: 0.744058 + 0.00712864
[19]	cv_agg's auc: 0.744137 + 0.00719492
[20]	cv_agg's auc: 0.744211 + 0.00744536
[21]	cv_agg's auc: 0.744376 + 0.00742565
[22]	cv_agg's auc: 0.744373 + 0.00754199
[23]	cv_agg's auc: 0.744343 + 0.00745782
[24]	cv_agg's auc: 0.744456 + 0.00748869
[25]	cv_agg's auc: 0.744576 +

(0.74558569494307, 787)

In [ ]:
model = lgb.train(lgb_parameters_st , lgb_train_st , lgb_st_rounds, categorical_feature= cat_cols)
pd.Series(model.predict(test_X_st)).to_csv('submission_Staking.csv',header = True, index_label = True)

lb : 0.764913

# The End